In [1]:
import matplotlib.pyplot as plt
import matplotlib
from typing import Tuple
from functools import cache
import itertools
import scipy.special
import scipy.stats
import numpy as np

%matplotlib inline
matplotlib.rcParams['font.size'] = 20
matplotlib.rcParams['figure.figsize'] = (16,9)
matplotlib.rcParams['savefig.bbox'] = 'tight'

# An implementation of artifact statistics
This code aims to demonstrate the ideas and maths shown in the following paper: [https://www.overleaf.com/read/nvxmkdpqjprj](https://www.overleaf.com/read/nvxmkdpqjprj)

I offer three contributions:

1. Very fast implementation of GenshinOptimizer's existing Roll Probability Calculator. frzyc noted that it's somewhat straining the front-end, so I've made a cache-based implementation that runs in microseconds per query.
2. A method that, given an optimization target and a current build, finds the artifact in the inventory that has the greatest chance of improving your build.
3. A method to evaluate any particular build by scoring how difficult it would be to improve any item of the build.

In [2]:
@cache
def nk3(n, k):
    return sum([scipy.special.binom(n, i) * scipy.special.binom(n, k-2*i) for i in range(k//2+1)])

@cache
def mu(a, j):
    return sum([nk3(j, i-7*j) for i in range(a, 10*j+1)]) / (4**j)

@cache
def multinom(j1, j2=None, j3=None, j4=None, N=5):
    # The standard multinomial distribution. Below logic handles variable arguments
    if j2 is None:
        rem = N-j1
        rv = scipy.stats.multinomial(N, [1/4, 3/4])
        return rv.pmf([j1, rem])
    
    if j3 is None:
        rem = N-j1-j2
        rv = scipy.stats.multinomial(N, [1/4, 1/4, 2/4])
        return rv.pmf([j1, j2, rem])
    
    if j4 is None:
        rem = N-j1-j2-j3
        rv = scipy.stats.multinomial(N, [1/4, 1/4, 1/4, 1/4])
        return rv.pmf([j1, j2, j3, rem])
    
    if j1+j2+j3+j4 != N:
        return 0
    rv = scipy.stats.multinomial(N, [1/4, 1/4, 1/4, 1/4])
    return rv.pmf([j1, j2, j3, j4])

## Type 1 Query (Contribution 1)
The type 1 query behaves as:
$$ P(A_1\geq a_1 \land A_2\geq a_2\land\cdots)$$

This is already implemented in frzyc's GenshinOptimizer, but issues regarding its computational load have been brought up. The following method I present is quite fast; with appropriate caching it reaches about 200µs per loop. (around .2s for 1000 queries)

----

Can be further optimized:
- iterate with nested loops i=0..N; j=0..(N-i); k=0..(N-i-j) rather than the cartesian products
- check for mu=0 conditions, whenever $a_i> 10(j_i+b)$. Skip computation of those terms.
- wrap the cached functions in a preprocessor to reduce required cache size
  - mu=1 whenever $a_i \leq 7(j_i+b)$
  - sort inputs to `multinom`, discard any zeros in the input
- sort the inputs (a1, a2, ...) and additionally cache `prob1` based on their inputs

In [3]:
# P(A1 >= a1 AND A2 >= a2)
def prob1(a1, a2=None, a3=None, a4=None, N=5, b=1):
    if a2 is None:
        return sum([multinom(j1, N=N) * mu(a1, j1+b) for j1 in range(N+1)])
    
    if a3 is None:
        ret = 0
        for j1, j2 in filter(lambda x: sum(x)<=N, itertools.product(range(N+1), repeat=2)):
            ret += multinom(j1, j2, N=N) * mu(a1, j1+b) * mu(a2, j2+b)
        return ret
    
    if a4 is None:
        ret = 0
        for j1, j2, j3 in filter(lambda x: sum(x)<=N, itertools.product(range(N+1), repeat=3)):
            ret += multinom(j1, j2, j3, N=N) * mu(a1, j1+b) * mu(a2, j2+b) * mu(a3, j3+b)
        return ret
    
    ret = 0
    for j1, j2, j3 in filter(lambda x: sum(x)<=N, itertools.product(range(N+1), repeat=3)):
        j4 = N-j1-j2-j3
        ret += multinom(j1, j2, j3, j4, N=N) * mu(a1, j1+b) * mu(a2, j2+b) * mu(a3, j3+b) * mu(a4, j4+b)
    return ret


In [4]:
%timeit prob1(*np.random.randint(20, size=4))

156 µs ± 6.94 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Type 1 Query on existing artifact
Existing artifacts reduce to a special case of the previous query. Since they're practically the same query, the runtime is essentially the same, and is faster if `rolls_left` is smaller.

In [5]:
def p1_existing(requirements, existing, rolls_left):
    transformed_requirements = []
    for ai, az in zip(requirements, existing):
        transformed_requirements.append(ai - az)
    
    return prob1(*transformed_requirements, N=rolls_left, b=0)

## Type 2 Query
The type 2 query behaves as:
$$ P(k_1A_1 + k_2A_2 + \cdots\geq a^*)$$

There are many reasons why this kind of query is interesting. For example, suppose I want my artifact to roll 100 EM or 15% crit rate, or anywhere in between. So some acceptable results would be:
- 100 EM, 0% cr
- 90 EM, 1.5% cr
- 80 EM, 3% cr
- 50 EM, 7.5% cr
- 20 EM, 12% cr
- 0 EM, 25% cr

Converting to the language of substat rolls, we're asking for 42.9 IVs in EM or 38.6 IVs in CR. Writing with linear coefficients, we can roughly phrase the problem as:
$$ \begin{array}{ccc}k_1 = 0.9 & k_2 = 1 & a^* = 38.6 \end{array} $$
$$ P(0.9A_1 + A_2 \geq 38.6) $$


The problem becomes rather nontrivial even for the combination of only two artifacts, especially when they have different weights.

In [6]:
# Returns list of (v, p(v)) which represent exact distribution of P(k1A1 + ... = v)
def exact(ks, js):
    if len(ks) == 0:
        return np.array([[0, 1]])
    
    k, j = ks[-1], js[-1]
    pr1 = np.array([(k*i, nk3(j, i-7*j) / 4**j) for i in range(7*j, 10*j+1)])
    pr2 = exact(ks[:-1], js[:-1])
    
    res = {}
    for v1, p1 in pr1:
        for v2, p2 in pr2:
            res[v1 + v2] = res.get(v1+v2, 0) + p1*p2
    return np.array([(k, v) for k, v in res.items()])

# Returns the probability the sum total value of k1A1 + ... exceeds thr.
# We can also guarantee some % error threshold, at the cost of computation time. Set err_thresh=0 to solve exactly.
def pnj(thr, kjs, err_thresh=None):
    # increase all j's by b    
    ks = kjs[0]
    js = (kjs[1]).astype(int)

    # Check for simple situations
    if thr <= 7 * np.sum(ks * js):
        return 1
    if thr > 10 * np.sum(ks*js):
        return 0
        
    # Gaussian estimate
    mu = 8.5 * np.sum(ks * js)
    var = 5/4 * np.sum(ks*ks*js)        
    p_est = scipy.special.erfc((thr - mu) / np.sqrt(2 * var)) / 2
    
    if err_thresh is None:
        return p_est

    # error estimate of Gaussian
    err = np.amax(ks) / (2 * np.sqrt(2 * var))
    max_err = err / p_est
    if max_err <= err_thresh:
        return p_est
        
    # Partially exact formulation
    ktarg = err_thresh * p_est * (2 * np.sqrt(2 * var))
    approx_select = ks < ktarg
    ext = exact(ks[~approx_select], js[~approx_select])
    
    # Construct partial approximate distribution
    ks, js = ks[approx_select], js[approx_select]
    mu = 8.5 * np.sum(ks * js)
    var = 5/4 * np.sum(ks*ks*js)
    if var > 0:
        p_est = lambda x: scipy.special.erfc((x - mu) / np.sqrt(2 * var)) / 2
    else:
        p_est = lambda x: 1 if x < 0 else 0
    
    # convolve exact distribution with approximate distribution
    ptot = 0
    for v, p in ext:
        ptot += p * p_est(thr - v)
    return ptot

def subsum(z):
    for i in range(z+1):
        for j in range(z-i+1):
            for k in range(z-i-j+1):
                yield (i, j, k, z-i-j-k)

def prob2(thresh, ks = (1), N=5, b=1, err=None):
    reps = 0
    ks = np.array(ks)
    
    ptot = 0
    if len(ks) == 4:
        it = subsum(N)
    else:
        it = filter(lambda x: sum(x) <= N, itertools.product(range(N+1), repeat=len(ks)))
    for js in it:
        js2 = np.array(js) + b
        mn = multinom(*js, N=N)
        if mn == 0: continue
        if thresh <= 7*np.sum(ks * js2):
            ptot += mn
            continue
        elif thresh > 10*np.sum(ks * js2):
            continue

        tmp = pnj(thresh, (ks, js2), err_thresh=err)
        ptot += mn * tmp
    return ptot

A note on the `err` paramter. I typically leave it unconstrained because my approximation is constructed very well. I've also found a way to mathematically guarantee certain error bounds, which naturally makes the computation time suffer.

For example, we could let `err=0.1` to guarantee that the returned result is within 10% of the true probability, or even `err=0` to force the program to find the exact probability.

In [7]:
print(f'Approximate P(100EM or 15CR): {prob2(38.6, [.9, 1])}')
print(f'Exact       P(100EM or 15CR): {prob2(38.6, [.9, 1], err=0)}')

Approximate P(100EM or 15CR): 0.4258741547179381
Exact       P(100EM or 15CR): 0.4154639244079591


In [8]:
%timeit prob2(38.6, [.9, 1])

627 µs ± 44.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%timeit prob2(38.6, [.9, 1, .2, 1], err=0)

11.3 ms ± 407 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit prob2(38.6, [.9, 1, .2, 1])

906 µs ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Type 2 Query on existing artifact
Same as with Type 1 queries, we just need to shift the threshold.

In [11]:
def p2_existing(thresh, ks, existing, rolls_left):
    new_th = thresh
    for ki, vi in zip(ks, existing):
        new_th -= ki*vi
    
    return prob2(new_th, ks, N=rolls_left, b=0)

## Connection to the Damage Formula
The following couple blocks set up my damage evaluation schema, where I've manually filled in some standard(ish) values.

All the percent- values are multiplied by 1000 for some reason.

In [12]:
import artifact2
import damage2
import random
from util.common import statnames, statmap, slotnames

In [21]:
import importlib
importlib.reload(damage2)

<module 'damage2' from '/Users/albertxu/PycharmProjects/playground/Genshin/damage2.py'>

In [13]:
# Stats of lvl 90 Diona w/ protype crescent
diona_stats = np.zeros(len(statnames))
diona_stats[statmap['HP']] = 9570
diona_stats[statmap['BaseATK']] = 722
diona_stats[statmap['ATK%']] = 413
diona_stats[statmap['DEF']] = 601
diona_stats[statmap['CR']] = 50
diona_stats[statmap['CD']] = 500
diona_stats[statmap['ER']] = 1000
diona_stats[statmap['Cryo']] = 240

In [14]:
# The artifacts I have equipped
flower = np.zeros(len(statnames))
flower[statmap['HP']] = 4780
flower[statmap['ATK']] = 33
flower[statmap['DEF']] = 16
flower[statmap['EM']] = 82
flower[statmap['CD']] = 140

feather = np.zeros(len(statnames))
feather[statmap['ATK']] = 311
feather[statmap['ER']] = 162
feather[statmap['DEF']] = 37
feather[statmap['CR']] = 31
feather[statmap['CD']] = 192

sands = np.zeros(len(statnames))
sands[statmap['EM']] = 187
sands[statmap['HP']] = 538
sands[statmap['DEF']] = 39
sands[statmap['CR']] = 124
sands[statmap['CD']] = 78

cup = np.zeros(len(statnames))
cup[statmap['Cryo']] = 466
cup[statmap['HP%']] = 47
cup[statmap['ATK%']] = 111
cup[statmap['ER']] = 220
cup[statmap['EM']] = 16

hat = np.zeros(len(statnames))
hat[statmap['CR']] = 311
hat[statmap['HP']] = 299
hat[statmap['CD']] = 155
hat[statmap['ATK%']] = 105
hat[statmap['EM']] = 56

In [24]:
12170 * 1.65

20080.5

In [25]:
dmg = damage2.NormalDmg(2.23) * damage2.CritMult() * damage2.VapeMelt(1.5) * damage2.DmgBonus('Cryo')
stats = diona_stats + flower + feather + sands + cup + hat
print(f'Current damage per charged shot: {dmg.eval(stats)[0]}')

Current damage per charged shot: 20761.071160967404


## Queries for replacing an artifact (Contribution 2)
Let's say I have a couple candidate artifacts to upgrade. All of them are EM sands, and they are all at lvl 0.

In [27]:
def to_stat(subs):
    st = np.zeros(len(statnames))
    for k, v in subs.items():
        st[statmap[k]] = v * artifact2.sub_vals[k] / 10
    return st

# Want to beat this number
sands0 = np.zeros(len(statnames))
sands0[statmap['EM']] = 187

stats = diona_stats + flower + feather + sands0 + cup + hat
subs_orig = {'HP': 18, 'DEF': 17, 'CR': 32, 'CD': 10}
dmg0 = dmg.eval(stats + to_stat(subs_orig))[0]

# Finds linear approximation of 4x4 Hessian
def appx_hessian2(H, k=1):
    a = np.sum(H) + np.trace(H)
    b = np.sum(H, axis=0) + np.diag(H)
    
    lhs = np.zeros(5)
    lhs[1:] = k / 7 * (a + 2*b)
    lhs[0] = k / 6 * a
    
    rhs_inv = 1 + np.eye(5)
    rhs_inv[:,0] = -6
    rhs_inv[0,:] = -k
    rhs_inv[0,0] = 5*k
    gv = rhs_inv @ lhs
    return gv[0], gv[1:]

# ixs must be length 3 or 4.
def linearize(statz, ixs, rolls=5):
    v, g, h = dmg.eval(statz)
    ix_names = [statnames[ix] for ix in ixs]
    scale = np.array([artifact2.sub_vals[k]/10 for k in ix_names])
    
    gnorm = g[ixs] * scale
    hnorm = h[ixs][:,ixs] * np.outer(scale, scale)
    
    if len(ixs) == 3:
        # force the thing to be 4x4.
        gnorm = np.append(gnorm, 0)
        hnorm = np.pad(hnorm, [0,1])
        rolls_left = 4
    
    c0, h_lin = appx_hessian2(hnorm, k=10*rolls)
    return v + c0, gnorm + h_lin / 2

def eval_potential(new_subs, rolls=0):
    rolls_left = 5 - rolls
    if len(new_subs) == 3:
        rolls_left = 4
    
    ix = [statmap[k] for k in new_subs]
    c, m = linearize(stats + to_stat(new_subs), ix, rolls_left)
    return prob2(dmg0 - c, m, N=rolls_left, b=0, err=None)

### Testing the maths
The validity of my maths are evaluated by upgrading some un-upgraded artifact 10,000 times and comparing the predicted probability with the observed probability.

The runtime of evaluating the potential of an un-upgraded artifact suffers a little bit. I don't know what exactly counts as acceptible, but it's around the 2ms mark.

In [28]:
def upgrade_artifact(subs_init, nn=5):
    up_locs = np.random.randint(4, size=nn)
    up_vals = np.random.randint(7, 11, size=nn)
    
    new_subs = {}
    for i, k in enumerate(subs_init):
        new_subs[k] = up_vals[up_locs == i].sum() + subs_init[k]
    return new_subs

def check_probability(subs, N=1000):
    num_ups = 5 - (4 - len(subs))
    test_vals = [dmg.eval(stats + to_stat(upgrade_artifact(subs, nn=num_ups)))[0] for _ in range(N)]
    return np.count_nonzero(np.array(test_vals) > dmg0) / N

In [29]:
# Pretend all of these are unupgraded EM main-stat sands.
sub1 = {'CD': 7, 'HP': 9, 'ATK%': 8, 'ATK': 7}
sub2 = {'CD': 8, 'CR': 9, 'HP': 10, 'ATK': 7}
sub3 = {'CD': 10, 'CR': 10, 'ATK%': 9}
sub4 = {'ATK': 8, 'DEF': 10, 'ATK%': 9, 'HP': 9}
sub5 = {'ATK': 7, 'ATK%': 10, 'CR': 8}

In [30]:
print(f'Math predictions:\t{eval_potential(sub4)}')
print(f'Simulated Value:\t{check_probability(sub4, N=10000)}')

Math predictions:	0.04525003754460953
Simulated Value:	0.0447


In [31]:
%timeit eval_potential(sub1)

1.95 ms ± 42.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Querying "completeness" of artifact set (Contribution 3)
We can design another query type that gives the expected amount of time it would take to improve upon your current set's total DPS. 

For each slot (flower, feather, etc.) and main stat (ATK%, EM, HP%, etc.) we can compute the probability that a random new artifact, once upgraded to max, will improve upon the current set's total DPS. Then we can take the weighted average of these probabilities to be the total probability that a new artifact will improve the set's DPS. The expected time we would need to farm to improve DPS is then the inverse of the probability.

In [73]:
prob_subs_summary = {
0: {
    (4, 6, 6, 6): 31833 / 2351440, (3, 6, 6, 6): 35150679 / 3618174560,
    (4, 4, 6, 6): 12247 / 1492260, (3, 4, 6, 6): 6395431221 / 1085396703776, (3, 3, 6, 6): 588789 / 139160560,
    (4, 4, 4, 6): 4249 / 852720, (3, 4, 4, 6): 83731933 / 23392170340, (3, 3, 4, 6): 53217 / 20691880,
    (4, 4, 4, 4): 1 / 330, (3, 4, 4, 4): 17447 / 8009760, (3, 3, 4, 4): 50647 / 32339406,
},
6: {
    (4, 4, 6, 6): 222877 / 12932920, (3, 4, 6, 6): 1724981913 / 140744203600,  (3, 3, 6, 6): 80433 / 9225944,
    (4, 4, 4, 6): 2407 / 235144, (3, 4, 4, 6): 15233623 / 2090714400, (3, 3, 4, 6): 3427272 / 660607675,
    (4, 4, 4, 4): 128 / 20995, (3, 4, 4, 4): 35624 / 8200647, (3, 3, 4, 4): 7813 / 2523276,
},
4: {
    (4, 6, 6, 6): 29 / 1309, (3, 6, 6, 6): 4745547 / 300284600,
    (4, 4, 6, 6): 1475 / 111384, (3, 4, 6, 6): 1824571 / 193040100, (3, 3, 6, 6): 92097 / 13649300,
    (4, 4, 4, 6): 589 / 74256, (3, 4, 4, 6): 1979168149 / 349325364960, (3, 3, 4, 6): 16088 / 3974355,
    (4, 4, 4, 4): 1 / 210, (3, 4, 4, 4): 27001 / 7931616, (3, 3, 4, 4): 70339 / 28893744,
},
3: {
    (4, 6, 6, 6): 106873344 / 5489226575, (3, 6, 6, 6): 5262165 / 378263704,
    (4, 4, 6, 6): 39435776 / 3375362925, (3, 4, 6, 6): 66962957661 / 8017134743800,
    (4, 4, 4, 6): 33193984 / 4725508095, (3, 4, 4, 6): 151137649 / 30075647580,
    (4, 4, 4, 4): 2048 / 483923, (3, 4, 4, 4): 2368048 / 781535645,
}}

def p_subs(vs, main_type):
    srch_key = tuple(sorted(map(lambda s: artifact2.substat.distr[s], vs)))
    if main_type not in artifact2.substat.distr:
        return prob_subs_summary[0][srch_key]
    return prob_subs_summary[artifact2.substat.distr[main_type]][srch_key]

In [74]:
stats = diona_stats + flower + feather + cup + hat
dmg0 = dmg.eval(stats + sands)[0]
def prob_beat(main, subs, p3sub=.8):
    ix = [statmap[s] for s in subs]
    sand0 = np.zeros(len(statnames))
    sand0[statmap[main]] = artifact2.main_vals[main]
    c, m = linearize(stats + sand0, ix, rolls=9)
    pp3sub = p3sub * prob2(dmg0 - c, m, N=4, b=1, err=None)
    pp4sub = (1-p3sub) * prob2(dmg0 - c, m, N=5, b=1, err=None)
    return pp3sub + pp4sub

In [75]:
artis = [flower, feather, sands, cup, hat]
def evaluate_slot(slot):
    global dmg0
    global stats
    stats = diona_stats + sum([a for i, a in enumerate(artis) if i != slot])
    dmg0 = dmg.eval(stats + artis[slot])[0]
    
    ptot = 0
    main_distr = artifact2.mainstat[slot].distr
    main_denom = sum(main_distr.values())
    for main, v_main in main_distr.items():
        p_main = v_main / main_denom
        sub_distr = artifact2.substat
        sub_distr = sub_distr.remove(main)
        for c in itertools.combinations(sub_distr.k, 4):
            prb = p_main * p_subs(c, main)
            ptot += prb * prob_beat(main, c, p3sub=.8)
    return ptot

### Evaluating
The function `evaluate_slot` returns an estimate of the probability that a random new flower/feather/etc. will beat the current set's total damage value. It's a rather slow function, taking a couple seconds to run depending on the number of options that the main stat can take on. Evaluating all 5 artifact slots takes about 10 seconds to run in total on my laptop.

In [76]:
for i in range(5):
    print(f'{slotnames[i]} - {evaluate_slot(i)}')

Flower - 0.03970803290118193
Feath - 0.28305707787509365
Sands - 0.04127033545386591
Cup - 0.024475889785761824
Hat - 0.0006308605765661203


In [35]:
%timeit evaluate_slot(3)

4.38 s ± 701 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can compare the above predictions to estimates of the actual value by making random artifacts below, using Monte-Carlo approach.

The (above) estimates are off by imo an acceptable amount, typically somewhere within 1% of the (below) true values.

In [70]:
def evalboi(slot, N=1000):
    stats = diona_stats + sum([a for i, a in enumerate(artis) if i != slot])
    dmg0 = dmg.eval(stats + artis[slot])[0]
    
    num_exc = 0
    for _ in range(N):
        a = artifact2.make_arti(slot=slot).tostat()
        dmg2 = dmg.eval(stats + a)[0]
        if dmg2 > dmg0:
            num_exc += 1
    return num_exc / N

In [71]:
for i in range(5):
    print(f'{slotnames[i]} - {evalboi(i, N=15000)}')

Flower - 0.05313333333333333
Feath - 0.29086666666666666
Sands - 0.046066666666666665
Cup - 0.025333333333333333
Hat - 0.0010666666666666667


In [72]:
for i in range(5):
    print(f'{slotnames[i]} - {evalboi(i, N=15000)}')

Flower - 0.05006666666666667
Feath - 0.29973333333333335
Sands - 0.04273333333333333
Cup - 0.0232
Hat - 0.001
